In [4]:
## TEMP for development only before moving into .py files

import os
import pandas as pd
import formula_page_generator
import utilities
import topics


website_creator_dir = os.getcwd()
docs_dir = utilities.get_docs_path(website_creator_dir)
formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
syllabus_file_path = (website_creator_dir + os.path.sep +
                        'syllabus_topics.csv')
sort_orders_df = pd.read_csv(order_file_path)
syllabus_df = pd.read_csv(syllabus_file_path)
formulas_input_df = pd.read_csv(formula_file_path)
formula_sheet_items = (
    formula_page_generator.get_formulas_on_formula_sheet(formulas_input_df))
formula_proof_required_items = (
    formula_page_generator.get_formulas_where_proofs_required(formulas_input_df))

utilities.delete_directory_if_it_exists(docs_dir)

# Create formula-related markdown documents
formulas_df = formula_page_generator.get_formulas_df(
    formulas_input_df, syllabus_df, sort_orders_df)
formula_page_generator.create_formulas_content(
    formulas_df, formula_sheet_items, formula_proof_required_items,
    sort_orders_df, docs_dir)
formula_page_generator.create_calculus_summary_files(
    formulas_df, formula_sheet_items, formula_proof_required_items,
    sort_orders_df, docs_dir)
formula_page_generator.create_financial_summary(
    formulas_df, formula_sheet_items, formula_proof_required_items,
    sort_orders_df, docs_dir)

# Generate topic pages
formulas_by_topic_df = formula_page_generator.get_formulas_by_topic_df(
    formulas_input_df, syllabus_df)

topics.create_topic_content(formulas_by_topic_df, formula_sheet_items,
                            formula_proof_required_items, sort_orders_df,
                            docs_dir)


In [1]:
import os
import utilities
from maths_objects import Formulas, WebPageHierarchies

website_creator_dir = os.getcwd()
docs_dir = utilities.get_docs_path(website_creator_dir)
formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
syllabus_file_path = (website_creator_dir + os.path.sep +
                        'syllabus_topics.csv')

page_sort_orders = WebPageHierarchies(order_file_path)
# school_formulas = Formulas(formula_file_path, syllabus_file_path,
#                            page_sort_orders)

In [7]:
# df = page_sort_orders.to_dataframe()


def fn_filter(item):
    return item['Level_1']=='Topics'

page_sort_orders.filter_by_function(
    lambda x: x['Level_1'] == 'Formulas').to_dataframe()


# return_row = df.apply(fn_filter, axis=1)
# display(df[return_row])



,Level_0,Level_1,Level_2,Level_3,Level_4,Level_5
8,NSW,Formulas,NaN,NaN,NaN,NaN
9,NSW,Formulas,By year,NaN,NaN,NaN
10,NSW,Formulas,By year cumulative,NaN,NaN,NaN
11,NSW,Formulas,By year,Year 9,NaN,NaN
12,NSW,Formulas,By year,Year 11 Adv,NaN,NaN
13,NSW,Formulas,By year,Year 12 Adv,NaN,NaN
14,NSW,Formulas,By year,Year 12 Ext 1,NaN,NaN
15,NSW,Formulas,By year,Year 9,Surds,NaN
16,NSW,Formulas,By year,Year 9,Indices,NaN
17,NSW,Formulas,By year,Year 12 Adv,Summaries,Calculus


In [34]:

# Incorporate below in index file creation to make it clearer how levels are 
# iterated.  Can I improve further by creating an iterator.  In the 
# Web page hierarchy class?

dir_levels = school_formulas.by_year_dirs()
current_dir_levels = dir_levels
for column in reversed(dir_levels.columns):
    display(current_dir_levels)
    current_dir_levels = current_dir_levels.drop(columns=column)

,State,Formula subcategory 1,Formula subcategory 2,Subject
0,NSW,Formulas,By Year,Year 9
9,NSW,Formulas,By Year,Year 11 Adv
21,NSW,Formulas,By Year,Year 12 Adv
42,NSW,Formulas,By Year,Year 12 Ext 1


,State,Formula subcategory 1,Formula subcategory 2
0,NSW,Formulas,By Year
9,NSW,Formulas,By Year
21,NSW,Formulas,By Year
42,NSW,Formulas,By Year


,State,Formula subcategory 1
0,NSW,Formulas
9,NSW,Formulas
21,NSW,Formulas
42,NSW,Formulas


,State
0,NSW
9,NSW
21,NSW
42,NSW


In [7]:
def create_index_files(base_dir, dir_levels,site_hierarchies,  
                       front_matter=None):
    """Creates _index.md files recursively at each level of the directory
    structures formed by joining base_dir string and each element of 
    dir_levels.  The content of each file is set based  on the content
    of the front_matter dictionary and optionally includes a weight
    (sort order) as stored in site_hierarhcies.

    Args:
        base_dir (string): The base direcory that together with dir_levels
            represent the directories where the _index files
            are created
        dir_levels (dataframe): The various directory levels that are joined
            to base_dir
        site_hierarchies (WebPageHierarhies): Object storing the web page 
            hierarchies and sort order
                        
        front_matter (dictionary, optional): Optional front matter text to be
            stored in the _index.md files.
            Defaults to None.
    """ 
    if front_matter is None:
        front_matter = {}
    for i in range(1, len(dir_levels.columns)+1):
        # Looping through various levels in directory structure
        subdirs_df = dir_levels.iloc[:, :i].drop_duplicates()
        for _, row in subdirs_df.iterrows():
            file_name = (base_dir + os.path.sep +
                         os.path.sep.join(row) + os.path.sep + '_index.md')
            if not os.path.isfile(file_name):
                # Don't overwrite file if it exists
                # Use a copy on each loop to start fresh and not
                # carry over any details from previos loop
                front_matter_to_write = front_matter.copy()
                if sort_orders_df is not None:
                    sort_order = lookup_list_in_df(
                        sort_orders_df, list(row.values))
                    if sort_order is not None:
                        front_matter_to_write['weight'] = sort_order + 1
                string_to_write = get_front_matter_string(
                    front_matter_to_write)
                with open(file_name, "w", encoding="utf-8") as text_file:
                    text_file.write(string_to_write)

1

In [4]:
mydict  = {1:7, 2:3}

mydict[1]

7